In [ ]:
import requests
import pandas as pd

BASE_URL = "https://www.fema.gov/api/open/v2/FimaNfipClaims"

def query_nfip_claims(filter_str=None, top=1000, skip=0, select=None):
    params = {
        "$top": top,
        "$skip": skip
    }
    if filter_str:
        params["$filter"] = filter_str
    if select:
        params["$select"] = select
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    results = data.get("FimaNfipClaims", [])
    return pd.DataFrame(results)

# Example usage: get the first 1000 claims in state FL
df = query_nfip_claims(filter_str="state eq 'FL'", top=5)
print(df.head())




   agricultureStructureIndicator                  asOfDate  \
0                          False  2025-09-09T00:00:00.000Z   
1                          False  2025-09-09T00:00:00.000Z   
2                          False  2025-09-09T00:00:00.000Z   
3                          False  2025-09-09T00:00:00.000Z   
4                          False  2025-09-09T00:00:00.000Z   

   basementEnclosureCrawlspaceType  policyCount crsClassificationCode  \
0                              NaN            1                  None   
1                              0.0            1                  None   
2                              NaN            1                  None   
3                              NaN            1                  None   
4                              NaN            1                  None   

                 dateOfLoss  elevatedBuildingIndicator  \
0  2009-03-19T00:00:00.000Z                       True   
1  2022-09-28T00:00:00.000Z                      False   
2  2022-09-28T

In [ ]:
df = query_nfip_claims(
    filter_str="state eq 'HI'",
    top=None,
    select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent"
)
print(df.head())

# Fill missing values with 0
df["amountPaidOnBuildingClaim"] = df["amountPaidOnBuildingClaim"].fillna(0)
df["amountPaidOnContentsClaim"] = df["amountPaidOnContentsClaim"].fillna(0)

# Compute total
df["totalPaid"] = df["amountPaidOnBuildingClaim"] + df["amountPaidOnContentsClaim"]
print(df.head())

  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    HI                        NaN                        NaN        2005   
1    HI                   54818.25                    9934.11        1983   
2    HI                   19892.31                       0.00        1982   
3    HI                    1275.42                       0.00        1985   
4    HI                    1414.67                       0.00        2024   

  floodEvent  
0       None  
1       None  
2   Flooding  
3       None  
4       None  
  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    HI                       0.00                       0.00        2005   
1    HI                   54818.25                    9934.11        1983   
2    HI                   19892.31                       0.00        1982   
3    HI                    1275.42                       0.00        1985   
4    HI                    1414.67                       0.00 

In [ ]:
# df = query_nfip_claims(
#     filter_str="state eq 'HI'",
#     top=None,
#     select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent,reportedZipCode"
# )
df = query_nfip_claims(
    filter_str="",
    top=None,
    select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent,reportedZipCode"
)
print(df.head())

# Fill missing values with 0
df["amountPaidOnBuildingClaim"] = df["amountPaidOnBuildingClaim"].fillna(0)
df["amountPaidOnContentsClaim"] = df["amountPaidOnContentsClaim"].fillna(0)

# Compute total
df["totalPaid"] = df["amountPaidOnBuildingClaim"] + df["amountPaidOnContentsClaim"]
print(df.head())

  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    TX                   50000.00                   20000.00        2017   
1    VA                   12843.93                    4428.58        2003   
2    FL                        NaN                        NaN        2009   
3    LA                        NaN                        NaN        2005   
4    LA                     206.05                       0.00        1995   

          floodEvent reportedZipCode  
0   Hurricane Harvey           77429  
1   Hurricane Isabel           23508  
2               None           33433  
3  Hurricane Katrina           70003  
4           Flooding           70094  
  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    TX                   50000.00                   20000.00        2017   
1    VA                   12843.93                    4428.58        2003   
2    FL                       0.00                       0.00        200

In [ ]:
BASE_URL = "https://www.fema.gov/api/open/v2/FimaNfipClaims"

def query_nfip_claims(filter_str=None, top=1000, skip=0, select=None):
    params = {
        "$top": top,
        "$skip": skip
    }
    if filter_str:
        params["$filter"] = filter_str
    if select:
        params["$select"] = select
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    results = data.get("FimaNfipClaims", [])
    return pd.DataFrame(results)

def get_payment_amounts(zip_code, date_of_loss):
  df = query_nfip_claims(
      filter_str=f"reportedZipCode eq '{zip_code}' and yearOfLoss eq {date_of_loss}",
      top=None,
      select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent"
  )
  df["amountPaidOnBuildingClaim"] = df["amountPaidOnBuildingClaim"].fillna(0)
  df["amountPaidOnContentsClaim"] = df["amountPaidOnContentsClaim"].fillna(0)

  # Compute total
  df["totalPaid"] = df["amountPaidOnBuildingClaim"] + df["amountPaidOnContentsClaim"]
  return df

df = get_payment_amounts(77429, 2017)
print(df)


    state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0      TX                   50000.00                   20000.00        2017   
1      TX                  174042.00                   46958.00        2017   
2      TX                  103420.65                    7789.53        2017   
3      TX                   18899.67                       0.00        2017   
4      TX                  123774.31                   44967.77        2017   
..    ...                        ...                        ...         ...   
995    TX                    5214.34                       0.00        2017   
996    TX                  110434.90                   29024.69        2017   
997    TX                   97659.02                   87583.63        2017   
998    TX                       0.00                       0.00        2017   
999    TX                  150530.25                   43568.34        2017   

           floodEvent  totalPaid  
0    Hurricane H

In [ ]:
def get_all_flood_event_counts():
    """Retrieve all unique floodEvent types and their counts across the dataset."""
    all_data = []
    top = 1000
    skip = 0

    while True:
        df = query_nfip_claims(top=top, skip=skip, select="floodEvent")
        if df.empty:
            break
        all_data.append(df)
        skip += top

    full_df = pd.concat(all_data, ignore_index=True)
    counts = full_df["floodEvent"].value_counts(dropna=False)
    return counts

# Example usage
flood_event_counts = get_all_flood_event_counts()
print(flood_event_counts)